# Escalonando os atributos

Neste exemplo, veremos como o escalonamento dos atributos afeta o desempenho do algoritmo do gradiente descendente.

In [1]:
# Import all necessary libraries
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import random
import urllib
# Retrieve file from github.
url = 'https://raw.githubusercontent.com/zz4fap/t319_aprendizado_de_maquina/main/misc/util_functions.py'
urllib.request.urlretrieve(url, filename='./util_functions.py')
import util_functions as util

In [2]:
# Always reset the pseudo-random numbers generator to a known value so that your results are always the same.
np.random.seed(1234)

## Gerando o conjunto de treinamento

In [3]:
# Number of pairs attribute/label.
N = 10000

In [4]:
# Attribute 1
x1 = 10.0*np.random.randn(N, 1) + 10.0

# Attribute 2
x2 = np.random.randn(N, 1) 

# Target function.
y = x1 + x2

# Noise.
w = np.random.randn(N, 1)

# Noisy version.
y_noisy = y + w

# Concatenate both column vectors to create the attribute matrix.
X = np.c_[x1, x2]

**OBS**.: Percebam que o atributo $x_1$ tem um intervalo de variação maior do que o de $x_2$, pois sua variância é maior, e, além disso, sua média não é 0 e sim, 10.

## Sem escalonamento dos atributos

Plotando os histogramas dos atributos x1 e x2

In [5]:
# Histograma dos atributos x1 e x2
util.plotHistogram(x1, x2)

AttributeError: module 'util_functions' has no attribute 'plotHistogram'

In [ ]:
# learning rate.
alpha = 0.001

# Maximum number of iterations.
n_iterations = 10000

# Execute the gradient descent algorithm to find a solution to this regression problem.
a, Jgd, a_hist, grad_hist, iteration = lab5.batchGradientDescent(X, y_noisy, alpha, n_iterations)

# Plot the error versus the number of iterations.
lab5.plotErroVersusIteration(Jgd, iteration)

In [ ]:
# Closed-form solution.
a_opt = np.linalg.pinv(np.transpose(X).dot(X)).dot(np.transpose(X).dot(y_noisy))

In [ ]:
# Generate points for plotting the cost-function surface without feature scaling.
J, A1, A2 = util.calculateErrorSurface(X, y_noisy)

In [ ]:
# Plot cost-function surface.
util.plotCostFunction(A1, A2, J, a_opt, a_hist, iteration)

In [ ]:
util.plotGradientHistory(grad_hist, iteration, x_max=50)

## 2. Padronizando os atributos

In [ ]:
# Import Class StandardScaler from module Preprocessing of 
# library sklearn responsible for standardizing the data.
from sklearn.preprocessing import StandardScaler

# Instantiate a Standard scaler.
stdScaler = StandardScaler()

# Standardize the features.
scaled_X = stdScaler.fit_transform(X)

scaled_x1 = scaled_X[:,0].reshape(N,1)
scaled_x2 = scaled_X[:,1].reshape(N,1)

In [ ]:
lab5.plotHistogram(scaled_x1, scaled_x2)

In [ ]:
# learning rate.
alpha = 0.3

# Maximum number of iterations.
n_iterations = 10000

# Execute the gradient descent algorithm to find a solution to this regression problem.
a, Jgd, a_hist, grad_hist, iteration = util.batchGradientDescent(scaled_X, y_noisy, alpha, n_iterations)

# Plot the error versus the number of iterations.
util.plotErroVersusIteration(Jgd, iteration)

In [ ]:
# Closed-form solution.
a_opt_pad = np.linalg.pinv(np.transpose(scaled_X).dot(scaled_X)).dot(np.transpose(scaled_X).dot(y_noisy))

In [ ]:
# Generate points for plotting the cost-function surface without feature scaling.
J, A1, A2 = util.calculateErrorSurface(scaled_X, y_noisy, -10.0, 30.0, -19.0, 21.0)

In [ ]:
# Plot cost-function surface.
util.plotCostFunction(A1, A2, J, a_opt_pad, a_hist, iteration)

In [ ]:
# Plot gradient history.
util.plotGradientHistory(grad_hist, iteration, x_max=10)

## 3. Normalizando os atributos

In [ ]:
# Import Class MinMaxScaler from module Preprocessing of library sklearn responsible for normalizing the data.
from sklearn.preprocessing import MinMaxScaler

# Instantiate a MinMax scaler.
minMaxScaler = MinMaxScaler()

# Standardize the features.
scaled_X = minMaxScaler.fit_transform(X)

scaled_x1 = scaled_X[:,0].reshape(N,1)
scaled_x2 = scaled_X[:,1].reshape(N,1)

In [ ]:
util.plotHistogram(scaled_x1, scaled_x2)

In [ ]:
# learning rate.
alpha = 0.3

# Maximum number of iterations.
n_iterations = 10000

# Execute the gradient descent algorithm to find a solution to this regression problem.
a, Jgd, a_hist, grad_hist, iteration = lab5.batchGradientDescent(scaled_X, y_noisy, alpha, n_iterations)

# Plot the error versus the number of iterations.
util.plotErroVersusIteration(Jgd, iteration)

In [ ]:
# Closed-form solution.
a_opt_norm = np.linalg.pinv(np.transpose(scaled_X).dot(scaled_X)).dot(np.transpose(scaled_X).dot(y_noisy))

In [ ]:
# Generate points for plotting the cost-function surface without feature scaling.
J, A1, A2 = util.calculateErrorSurface(scaled_X, y_noisy, -10.0, 94.0, -72.0, 30.0)

In [ ]:
# Plot cost-function surface.
util.plotCostFunction(A1, A2, J, a_opt_norm, a_hist, iteration)

In [ ]:
util.plotGradientHistory(grad_hist, iteration, x_max=60)